In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../20230505_Messages.csv")
df.head()

,Datetime,Message
0,2023-03-01 14:28:13,Please leave a one line intro when you join an...
1,2023-03-01 14:29:24,I'm Pranjal Mehta. Cofounded ePlane.ai (electr...
2,2023-03-01 14:29:26,"Hey folks,"
3,2023-03-01 14:29:50,I'm Nirant K. ML/LLM Consultant out of BLR. Mo...
4,2023-03-01 14:30:27,Hey


In [3]:
df["Message"][21]

'Looks like @919619491715 got a DM from our benevolent dictator 😂, welcome to the group'

In [4]:
# Group by week
df["Datetime"] = pd.to_datetime(df["Datetime"])
df['Week'] = df['Datetime'].dt.isocalendar().week
df['Date'] = df['Datetime'].dt.date

# Group by Date
daily_df = df.groupby('Date').agg({'Message': ' \n '.join}).reset_index()
daily_df = pd.DataFrame(daily_df)
len(daily_df)

# # Group by Week
# weekly_df = df.groupby('Week').agg({'Message': ' \n '.join}).reset_index()
# weekly_df = pd.DataFrame(messages_df)
# print(weekly)
# print(weekly_df["Message"][9])

63

In [5]:
daily_df["wc"] = daily_df["Message"].apply(lambda x: len(x.split()))

In [6]:
daily_df["wc"].describe()

count      63.000000
mean      900.492063
std       727.693703
min         8.000000
25%       286.500000
50%       695.000000
75%      1498.000000
max      2540.000000
Name: wc, dtype: float64

In [17]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

from functools import lru_cache
llm = ChatOpenAI(temperature=0)

In [18]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()

In [85]:
%%time
class SummarizeDay:
    def __init__(self, plain_text):
        self.prompt_template = """This is a chaotic Generative AI Group Chat transcript. Write detailed, exhaustive bullet point recap of topics discussed. Extract COMPLETE URL of web and social links with context. Please organise it into sections, only when needed:

{text}


Use Markdown. Add ## for section titles. TOPICS RECAP:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output

plain_text = daily_df["Message"][36]
sd = SummarizeDay(plain_text)
chain_output = sd.summarize_docs()
output_text = chain_output["output_text"]
print(output_text)

## Integration of chatGPT/LLMs into smart home devices
- Query about integration of chatGPT/LLMs into Alexa/Nest type of smart home devices
- Mention of tools that can do it, but openai key is a barrier to adoption

## Paddlespeech
- Query about anyone dabbling with paddlespeech

## ChatGPT talk by Anil Ananthaswamy
- Video of the talk by Anil Ananthaswamy on ChatGPT held at BIC is published
- Link: https://youtu.be/WF28ZwhUCc4

## Automatic1111 and DrawThings App
- Use of Automatic1111 on colab
- Recommendation to download DrawThings App on MacBook
- Suggestion to try smaller models for GPU memory issue
- Mention of Jay trying to get it running on local via Web GPU
- Image generation on windows took a lot of time
- Automatic1111 works well on Mac for simple text2image, but may have trouble with complex operations
- Mention of text-to-video taking off in the future

## MacBook and generative AI capabilities
- Discussion on buying a MacBook for generative AI capabilities
- Recommendatio

In [81]:
import re
WINDOW = 1
def extract_urls_with_context(text):
    lines = text.split('\n')
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    urls_with_context = []

    for idx, line in enumerate(lines):
        for match in url_pattern.finditer(line):
            start, end = match.span()
            prev_line = lines[idx - WINDOW] if idx > 0 else ""
            next_line = lines[idx + WINDOW] if idx < len(lines) - 1 else ""
            context = f"{prev_line}\n{line}\n{next_line}".strip()
            urls_with_context.append((match.group(), context))

    return urls_with_context


    urls_with_context = extract_urls_with_context(url)
    for url, context in urls_with_context:
        print(f"URL: {url}")
        print(f"Context: {context}\n")
        


class LinksContext:
    def __init__(self, plain_text):
        self.prompt_template = """For each URL, there is some context. Newlines may or may not be related to the link, but the message in the same link as link is related to the link.
        
{text}
        
Mention URL with context. Single bullet point:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name="gpt-4"), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output

plain_text = daily_df["Message"][54]
url_groups = extract_urls_with_context(plain_text)
for ug in url_groups:
    lc = LinksContext(ug[1])
    chain_output = lc.summarize_docs()
    output_text = chain_output["output_text"]
    print(output_text)

- https://twitter.com/Mascobot/status/1651022921056555008?t=3qozUieRAPdsnScv3XnQLw&s=19 - The tweet mentions that something was built in under 2 weeks and discusses the differences between Codex iterations and vanilla LLM, including GPT3.5-turbo and GPT4.
- https://www.reddit.com/r/StableDiffusion/comments/12yzd2a/google_researchers_achieve_performance/ - Discussion thread on ML on edge computing, with a mention of Stanford NLP.
- https://www.qualcomm.com/news/onq/2023/02/worlds-first-on-device-demonstration-of-stable-diffusion-on-android: Some earlier work from Qualcomm, "Wow!"
- PSA: Dedicated group for music, images, video: https://chat.whatsapp.com/GThJJhoF3cL7QCmrfIoY8J
- https://twitter.com/matthieurouif/status/1650904940036890626 - A Twitter post discussing the availability of an API for the GPT-4 multimodal model.
- https://minigpt-4.github.io/ or https://llava-vl.github.io/ - These URLs can be used instead of Photoroom as it doesn't have image understanding.
- https://minigpt-